Imports

1. Откройте файл с данными. Посмотрите где и в каком количестве есть пропуски. Строки, где пропуски по 2 и более признакам удалите (т.е. это строки под номерами 3 и 271).

2. Разделите весь набор данных на 2 части: часть без пропусков data и часть data_with_nan, где имеются пропуски в столбце "sex" 

3. Решите задачу классификации пингвинов по полу методом KNN на наборе данных data:

отделите целевой признак от нецелевых
разделите данные на части для обучения train и тестирования test
проведите кодирование категориальных признаков в train, а затем в test используя кодировщики, обученные на train
проведите нормирование нецелевых признаков в train, а затем в test используя нормировщик, обученный на train
проведите подбор гиперпараметров (количество соседей и метрика расстояния) в KNN
обучите классификатор с наилучшими гиперпараметрами на train
проверьте его качество на test

4. Используя обученный классификатор заполните пропуски пола в  data_with_nan. Предварительно надо

 у data_with_nan  удалить столбец "sex"; 
провести кодирование нецелевых признаков обученным  на train кодировщиком; 
провести нормировку обученным на train нормализатором. 
После этого предскажите обученным классификатором значение целевого признака на данных их data_with_nan

5. Объедините нецелевые столбцы из data_with_nan и предсказанные значения пола, выведите их. Сравните с заполнением, которое мы делали, ориентируясь на вес пингвина